In [1]:
import pandas as pd
import datetime as dt
import hvplot.pandas

In [2]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [3]:
coin_list = cg.get_coins_list()

coins = pd.DataFrame(coin_list)

coins

,id,symbol,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0-knowledge-network,0kn,0 Knowledge Network
3,0vix-protocol,vix,0VIX Protocol
4,0x,zrx,0x Protocol
...,...,...,...
10035,zynecoin,zyn,Zynecoin
10036,zynergy,zyn,Zynergy
10037,zyrri,zyr,Zyrri
10038,zyx,zyx,ZYX


In [4]:
tokens = [
    'GTC',
    'GIV',
    'ENS',
    'KLIMA',
    'RAD',
    'BRIGHT',
    'HONEY',
    'TEC',
]

In [5]:
coins = coins[coins['symbol'].str.contains('|'.join(tokens).lower())]
coins

,id,symbol,name
668,astra-dao,astradao,Astra DAO
1462,bright-token,bright,BrightID
1463,bright-union,bright,Bright Union
1938,coinradr,radr,CoinRadr
2200,cryptomeda,tech,Cryptomeda
2328,dappradar,radar,DappRadar
3083,ethereum-name-service,ens,Ethereum Name Service
3222,farmer-friends,frens,Farmer Friends
3459,frens-coin,frens,Frens Coin
3548,game,gtc,Game


In [6]:
token_ids = [
    'bright-token',
    'ethereum-name-service',
    'gitcoin',
    'giveth',
    'rad',
    'honey-finance',
    'token-engineering-commons',
]

exact_match_regex = ['^'+id+'$' for id in token_ids]

In [7]:
coins[coins['id'].str.match('|'.join(exact_match_regex).lower())]

,id,symbol,name
1462,bright-token,bright,BrightID
3083,ethereum-name-service,ens,Ethereum Name Service
3687,gitcoin,gtc,Gitcoin
3691,giveth,giv,Giveth
4128,honey-finance,honey,Honey Finance
7170,rad,rad,RAD
8739,token-engineering-commons,tec,Token Engineering Commons


In [8]:
token_id = 'bright-token'
quote = 'ETH'
days = 730

# Pull prices from coin gecko
prices = cg.get_coin_market_chart_by_id(token_id, quote, days)

# Little bit of clean up
prices['timestamp'] = [[p[0],p[0]] for p in prices['prices']]
prices = {a:[i[1] for i in b] for a,b in prices.items()}

# Make a dataframe time series
prices_df = pd.DataFrame(prices)

prices_df['token_id'] = token_id

prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000326,0.0,593.830427,1632009600000,bright-token
1,0.000322,0.0,80.248617,1632096000000,bright-token
2,0.000270,0.0,92.635804,1632182400000,bright-token
3,0.000268,0.0,50.559488,1632268800000,bright-token
4,0.000254,0.0,46.795054,1632355200000,bright-token
...,...,...,...,...,...
726,0.000022,0.0,8.871477,1694736000000,bright-token
727,0.000021,0.0,0.053504,1694822400000,bright-token
728,0.000022,0.0,4.189035,1694908800000,bright-token
729,0.000022,0.0,0.095688,1694995200000,bright-token


In [9]:
def get_price_history(token_id: str, quote: str, days: int) -> pd.DataFrame:
    # Pull prices from coin gecko
    prices = cg.get_coin_market_chart_by_id(token_id, quote, days)
    
    # Little bit of clean up
    prices['timestamp'] = [[p[0],p[0]] for p in prices['prices']]
    prices = {a:[i[1] for i in b] for a,b in prices.items()}
    
    # Make a dataframe time series
    prices_df = pd.DataFrame(prices)
    
    prices_df['token_id'] = token_id
    
    return prices_df

In [10]:
token_ids

['bright-token',
 'ethereum-name-service',
 'gitcoin',
 'giveth',
 'rad',
 'honey-finance',
 'token-engineering-commons']

In [11]:
df = get_price_history(token_ids[1], quote='ETH', days=730)
df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.009160,124926.516523,118492.529132,1636416000000,ethereum-name-service
1,0.009160,124926.516523,118492.529132,1636502400000,ethereum-name-service
2,0.016197,258907.913675,312098.635595,1636588800000,ethereum-name-service
3,0.012147,216011.170488,179993.387637,1636675200000,ethereum-name-service
4,0.011598,211964.062751,84745.840891,1636761600000,ethereum-name-service
...,...,...,...,...,...
675,0.004624,119248.131848,8981.062749,1694736000000,ethereum-name-service
676,0.004676,120552.667666,6229.560038,1694822400000,ethereum-name-service
677,0.004729,121878.159492,4267.844934,1694908800000,ethereum-name-service
678,0.004550,117156.527820,4122.771385,1694995200000,ethereum-name-service


In [12]:
data = []
for token_id in token_ids:
    print(f"Fetching data for token_id: {token_id}")
    df = get_price_history(token_id, quote='ETH', days=730)
    data.append(df)

print(f"Fetching data for ETH/USD")
df = get_price_history('ethereum', quote='usd', days=730)
data.append(df)

Fetching data for token_id: bright-token
Fetching data for token_id: ethereum-name-service
Fetching data for token_id: gitcoin
Fetching data for token_id: giveth
Fetching data for token_id: rad
Fetching data for token_id: honey-finance
Fetching data for token_id: token-engineering-commons
Fetching data for ETH/USD


In [13]:
# Concatenate all of the data into one dataframe
prices_df = pd.concat(data)

# Create datetime column from the timestamp
prices_df['timestamp'] = pd.to_datetime(prices_df['timestamp'], unit='ms')

prices_df

,prices,market_caps,total_volumes,timestamp,token_id
0,0.000326,0.000000e+00,5.938304e+02,2021-09-19 00:00:00,bright-token
1,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token
2,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token
3,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token
4,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token
...,...,...,...,...,...
726,1626.436341,1.954638e+11,8.399895e+09,2023-09-15 00:00:00,ethereum
727,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum
728,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum
729,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum


In [14]:
import hvplot.pandas

In [15]:
prices_df.hvplot.step(x='timestamp', y='prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (prices)

Create normalized prices.

In [16]:
prices_df['normalized_prices'] = prices_df['prices'] / prices_df.groupby('token_id')['prices'].transform('first')

In [17]:
prices_df.hvplot.step(x='timestamp', y='normalized_prices', by='token_id')

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:NdOverlay   [token_id]
   :Curve   [timestamp]   (normalized_prices)

Giveth has some anomolous behavior near the beginning. I'm going to remove the first 20 days of giveth.

In [49]:
import holoviews as hv

prices_df_adjusted = prices_df.copy(deep=True)

prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'] = prices_df_adjusted[prices_df_adjusted['token_id']=='giveth'].iloc[20:]

prices_df_adjusted['normalized_prices'] = prices_df_adjusted['prices'] / prices_df_adjusted.groupby('token_id')['prices'].transform('first')

In [50]:
plot = prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].hvplot.line(grid=True, rot=45, title='Public Goods Tokens Normalized against Ethereum Sept 15th 2021 - Sept 15th 2023',x='timestamp', y='normalized_prices', by='token_id', width=2200, height=1000, line_width=3) * hv.HLine(1)
plot

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [51]:
plot.opts(xticks=[(date, date.strftime('%B %Y')) for date in pd.date_range(start=prices_df_adjusted['timestamp'].min(), end=prices_df_adjusted['timestamp'].max(), freq='MS')])

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (normalized_prices)
   .HLine.I     :HLine   [x,y]

In [52]:
prices_df_adjusted['usd_normalized_prices'] = prices_df_adjusted.groupby('token_id')['normalized_prices'].transform(lambda x: x * prices_df_adjusted[prices_df_adjusted['token_id']=='ethereum']['normalized_prices'])

In [53]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].hvplot.line(title='Normalized Price Comparison Sept 15th 2021 - Sept 15th 2023',x='timestamp', y='usd_normalized_prices', by='token_id', width=2200, height=1000, line_width=3) * hv.HLine(1)

/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type
/home/ygg/.cache/pypoetry/virtualenvs/tec1pager-028XofSl-py3.10/lib/python3.10/site-packages/holoviews/core/data/pandas.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return dataset.data.dtypes[idx].type


:Overlay
   .NdOverlay.I :NdOverlay   [token_id]
      :Curve   [timestamp]   (usd_normalized_prices)
   .HLine.I     :HLine   [x,y]

In [93]:
prices_df_adjusted['returns'] = prices_df_adjusted.groupby('token_id')['normalized_prices'].transform(lambda x: x.bfill().pct_change()+1)

In [94]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,usd_normalized_prices,returns
0,0.000326,0.000000e+00,5.938304e+02,2021-09-19 00:00:00,bright-token,1.000000,1.000000,NaN
1,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token,0.986014,0.959635,0.986014
2,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token,0.826329,0.717779,0.838049
3,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token,0.821731,0.657874,0.994436
4,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token,0.778728,0.698423,0.947669
...,...,...,...,...,...,...,...,...
726,1626.436341,1.954638e+11,8.399895e+09,2023-09-15 00:00:00,ethereum,0.474514,0.225163,1.011631
727,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum,0.479036,0.229475,1.009529
728,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum,0.476843,0.227379,0.995423
729,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum,0.473359,0.224069,0.992694


In [104]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').agg(
    mean_return=('returns', 'mean'),
    std=('returns', 'std'),
    sharpe_ratio=('returns', lambda x: (x.mean()-1) / x.std()),
).sort_values('sharpe_ratio', ascending=False)
    

,mean_return,std,sharpe_ratio
token_id,,,
ethereum-name-service,1.000286,0.054224,0.005275
gitcoin,1.000081,0.079957,0.001007
giveth,0.999472,0.167297,-0.003159
bright-token,0.998903,0.076746,-0.014292
token-engineering-commons,0.998393,0.046764,-0.034371
honey-finance,0.997710,0.052322,-0.043759
rad,0.993804,0.087567,-0.070751


In [110]:
prices_df_adjusted[prices_df_adjusted['token_id'] != 'ethereum'].groupby('token_id').apply(lambda x: x['normalized_prices'].iloc[-1] / x['returns'].std()).sort_values(ascending=False)

token_id
ethereum-name-service        9.270571
token-engineering-commons    4.345136
honey-finance                2.736194
gitcoin                      2.593724
bright-token                 0.844375
giveth                       0.131900
rad                          0.047730
dtype: float64

In [91]:
prices_df_adjusted.groupby('token_id')['normalized_prices'].apply(lambda x: (x.iloc[-1] / x.std())).sort_values(ascending=False)

token_id
ethereum-name-service        1.782260
ethereum                     1.730367
token-engineering-commons    0.870732
gitcoin                      0.674122
bright-token                 0.442437
honey-finance                0.315426
giveth                       0.092631
rad                          0.017039
Name: normalized_prices, dtype: float64

In [80]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,usd_normalized_prices,returns
0,0.000326,0.000000e+00,5.938304e+02,2021-09-19 00:00:00,bright-token,1.000000,1.000000,NaN
1,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token,0.986014,0.959635,0.986014
2,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token,0.826329,0.717779,0.838049
3,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token,0.821731,0.657874,0.994436
4,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token,0.778728,0.698423,0.947669
...,...,...,...,...,...,...,...,...
726,1626.436341,1.954638e+11,8.399895e+09,2023-09-15 00:00:00,ethereum,0.474514,0.225163,1.011631
727,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum,0.479036,0.229475,1.009529
728,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum,0.476843,0.227379,0.995423
729,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum,0.473359,0.224069,0.992694


In [81]:
prices_df_adjusted.reset_index()['returns'].bfill().idxmax()

2308

In [82]:
prices_df_adjusted.iloc[4484]

prices                           3427.584262
market_caps              403306688199.629822
total_volumes             16278153590.132133
timestamp                2021-09-19 00:00:00
token_id                            ethereum
normalized_prices                        1.0
usd_normalized_prices                    1.0
returns                                  NaN
Name: 0, dtype: object

In [83]:
1.055 * 0.946

0.9980299999999999

In [84]:
prices_df_adjusted

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,usd_normalized_prices,returns
0,0.000326,0.000000e+00,5.938304e+02,2021-09-19 00:00:00,bright-token,1.000000,1.000000,NaN
1,0.000322,0.000000e+00,8.024862e+01,2021-09-20 00:00:00,bright-token,0.986014,0.959635,0.986014
2,0.000270,0.000000e+00,9.263580e+01,2021-09-21 00:00:00,bright-token,0.826329,0.717779,0.838049
3,0.000268,0.000000e+00,5.055949e+01,2021-09-22 00:00:00,bright-token,0.821731,0.657874,0.994436
4,0.000254,0.000000e+00,4.679505e+01,2021-09-23 00:00:00,bright-token,0.778728,0.698423,0.947669
...,...,...,...,...,...,...,...,...
726,1626.436341,1.954638e+11,8.399895e+09,2023-09-15 00:00:00,ethereum,0.474514,0.225163,1.011631
727,1641.934758,1.973216e+11,6.794504e+09,2023-09-16 00:00:00,ethereum,0.479036,0.229475,1.009529
728,1634.419885,1.964891e+11,4.301245e+09,2023-09-17 00:00:00,ethereum,0.476843,0.227379,0.995423
729,1622.478777,1.948948e+11,4.510503e+09,2023-09-18 00:00:00,ethereum,0.473359,0.224069,0.992694


In [85]:
prices_df_adjusted[prices_df_adjusted['token_id']=='token-engineering-commons']

,prices,market_caps,total_volumes,timestamp,token_id,normalized_prices,usd_normalized_prices,returns
0,0.001200,0.0,4.412721,2022-01-25 00:00:00,token-engineering-commons,1.000000,1.000000,NaN
1,0.001200,0.0,4.412721,2022-01-26 00:00:00,token-engineering-commons,1.000000,0.973247,1.000000
2,0.000953,0.0,2.566614,2022-01-27 00:00:00,token-engineering-commons,0.794553,0.690177,0.794553
3,0.001048,0.0,2.058658,2022-01-28 00:00:00,token-engineering-commons,0.873267,0.699134,1.099067
4,0.000959,0.0,0.155690,2022-01-29 00:00:00,token-engineering-commons,0.798987,0.716592,0.914940
...,...,...,...,...,...,...,...,...
597,0.000259,0.0,0.037099,2023-09-15 00:00:00,token-engineering-commons,0.215922,0.116496,0.979974
598,0.000251,0.0,0.072697,2023-09-16 00:00:00,token-engineering-commons,0.209519,0.112845,0.970346
599,0.000246,0.0,0.095854,2023-09-17 00:00:00,token-engineering-commons,0.205332,0.110398,0.980016
600,0.000263,0.0,0.008110,2023-09-18 00:00:00,token-engineering-commons,0.219207,0.114993,1.067573


In [86]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['normalized_prices'].agg('last').sort_values(ascending=False)

token_id
ethereum-name-service        0.502685
gitcoin                      0.207386
token-engineering-commons    0.203197
honey-finance                0.143162
bright-token                 0.064803
giveth                       0.022066
rad                          0.004180
Name: normalized_prices, dtype: float64

In [108]:
prices_df_adjusted[prices_df_adjusted['token_id']!='ethereum'].groupby('token_id')['returns'].agg('std').sort_values(ascending=True)

token_id
token-engineering-commons    0.046764
honey-finance                0.052322
ethereum-name-service        0.054224
bright-token                 0.076746
gitcoin                      0.079957
rad                          0.087567
giveth                       0.167297
Name: returns, dtype: float64